In [39]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
L=LaurentSeriesRing(QQ,'x', default_prec=300)

def polynomialdivision(dividend, divisor):
    q,r = dividend.maxima_methods().divide(divisor)
    return (q,r)
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNewStep(n,m): 
    f=J(n,m)
    fl=f.list()
    return (sum(c*x**i*(m**3*2**6)**i for i,c in enumerate(fl)))
def xjNew(n,m):
    return xjNewStep(n-1,m)
def xjNewCoefficient(n,m):
    if n>0:return (xjNew(n,m)-xjNew(n-1,m))/x^n
    if n==0:return 1
def xjNewTest(n,m):
    return D(xjNewStep(n-1,m))
def j(n,m):return L(xjNew(n,m)/x)

def H4strike(n,m):
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    poly=((S(numerator/denominator)^(1/(m-2))).O(n+1)).polynomial()
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def Hlambda4strike(n,m):
    jay=J(n+1,m)
    djay=x*derivative(jay,x) # bc variable is tau, not x; chain rule.
    numerator = djay^2
    denominator = jay*(jay-1)
    poly=((S(numerator/denominator)).O(n+1)).polynomial()
    polylist=poly.list()
    polydegree=poly.degree()
    answer=sum(polylist[k]*(x*m^3*2^6)^k for k in [0..polydegree])
    return answer

def polynomialCoefficient(n,poly):
    return poly.list()[n]
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
def quotientFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[0]
def remainderFOvrG(f,g) : 
    return (f.maxima_methods().divide(g))[1]
def factor(polynomial):
    fp=polynomial.factor()
    return fp
def numberOfFactors(polynomial):
    fp=polynomial.factor()
    return len(fp)
def factorK(polynomial,k):
    fp=factor(h)
    return fp[k-1]
def factorList(polynomial):
    answer=[]
    nf=numberOfFactors(polynomial)
    for k in [1..nf]:
        answer=answer+[factorK(polynomial,k)]
    return answer
def polynomialExponentList(polynomial):
    pel=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        pel=pel+[exponent]
    return pel
def polynomialFactorBaseList(polynomial):
    fbl=[]
    fp=polynomial.factor()
    lf=len(fp)
    for n in [0..lf-1]:
        part = fp[n]
        (base,exponent) = part
        fbl=fbl+[base]
    return fbl

def division(dividend, divisor) : 
    quotient=(dividend._maxima_().divide(divisor).sage())[0] 
    remainder=(dividend._maxima_().divide(divisor).sage())[1] 
    return (quotient,remainder)

def nieburTau(n):
# The contortions below sidestep mysterious (to me) error messages...
# nieburTau is an algorith for Ramanujan''s tau from the paper
# "A formula for Ramanujan's tau-function" by Douglas Niebur.
# (He spells tau with the Greek letter.)
    def step1(n,k):
        return 35*k^4-52*k^3*n+18*k^2*n^2
    def step2(n,k):
        return sigma(k)*sigma(n-k)
    def step3(n):
        ans=0
        for k in [1..n-1]:
            ans=ans+step1(n,k)*step2(n,k)
        return 24*ans
    return n^4*sigma(n)-step3(n)

In [2]:
poly=Hlambda4strike(5,3);
print(poly)
print(poly.degree())
print list(poly)

30240*x^5 + 17520*x^4 + 6720*x^3 + 2160*x^2 + 240*x + 1
5
[1, 240, 2160, 6720, 17520, 30240]


In [3]:
j(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

In [2]:
no=[]
polystrike=Hlambda4strike(12,3)
polystrikelist=polystrike.list()
for k in [1..12]:
    print "--------------------------------------------------------------------------"
    print [k,240*sigma(k,3),polystrikelist[k]]

--------------------------------------------------------------------------
[1, 240, 240]
--------------------------------------------------------------------------
[2, 2160, 2160]
--------------------------------------------------------------------------
[3, 6720, 6720]
--------------------------------------------------------------------------
[4, 17520, 17520]
--------------------------------------------------------------------------
[5, 30240, 30240]
--------------------------------------------------------------------------
[6, 60480, 60480]
--------------------------------------------------------------------------
[7, 82560, 82560]
--------------------------------------------------------------------------
[8, 140400, 140400]
--------------------------------------------------------------------------
[9, 181680, 181680]
--------------------------------------------------------------------------
[10, 272160, 272160]
-----------------------------------------------------------------------

In [5]:
diffs=[]
poly=Hlambda4strike(100,3)
plist=poly.list()
for k in [1..poly.degree()]:
    diffs=diffs+[240*sigma(k,3)-plist[k]]
print diffs

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [3]:
data=[]
import time
start=time.time()
for m in [3..503]:
    poly=Hlambda4strike(80,m)
    data=data+[[m,poly]]
    finish=time.time()
    print([m,poly.degree(),finish-start])
import pickle
wfile = open('/Users/barrybrent/31aug21no1.txt','w') # Hlambda4strike series 
s = pickle.dumps(str(data))
wfile.write(s)
wfile.close()

[3, 80, 0.7202610969543457]
[4, 80, 1.1610891819000244]
[5, 80, 1.9651191234588623]
[6, 80, 2.400374174118042]
[7, 80, 3.312373161315918]
[8, 80, 3.9834370613098145]
[9, 80, 4.970103025436401]
[10, 80, 5.605785131454468]
[11, 80, 6.579064130783081]
[12, 80, 7.4430530071258545]
[13, 80, 8.533699989318848]
[14, 80, 9.222656011581421]
[15, 80, 10.403258085250854]
[16, 80, 11.176814079284668]
[17, 80, 12.294270992279053]
[18, 80, 13.060512065887451]
[19, 80, 14.191358089447021]
[20, 80, 15.155831098556519]
[21, 80, 16.374508142471313]
[22, 80, 17.140748023986816]
[23, 80, 18.25441598892212]
[24, 80, 19.244920015335083]
[25, 80, 20.46130108833313]
[26, 80, 21.305041074752808]
[27, 80, 22.5768461227417]
[28, 80, 23.594935178756714]
[29, 80, 24.73885416984558]
[30, 80, 25.669538021087646]
[31, 80, 26.8523051738739]
[32, 80, 27.734791040420532]
[33, 80, 29.06714415550232]
[34, 80, 29.937602043151855]
[35, 80, 31.365036010742188]
[36, 80, 32.5512261390686]
[37, 80, 34.02478218078613]
[38, 80, 3

[287, 80, 397.2490351200104]
[288, 80, 398.67767119407654]
[289, 80, 400.4536371231079]
[290, 80, 401.82399106025696]
[291, 80, 403.5946991443634]
[292, 80, 405.0931851863861]
[293, 80, 406.7987611293793]
[294, 80, 408.2520830631256]
[295, 80, 410.02585315704346]
[296, 80, 411.4762740135193]
[297, 80, 413.26114201545715]
[298, 80, 414.5886781215668]
[299, 80, 416.38789916038513]
[300, 80, 418.0381281375885]
[301, 80, 419.8214991092682]
[302, 80, 421.0895540714264]
[303, 80, 422.8731460571289]
[304, 80, 424.3001410961151]
[305, 80, 426.11645007133484]
[306, 80, 427.541540145874]
[307, 80, 429.2655770778656]
[308, 80, 430.9122459888458]
[309, 80, 432.7716500759125]
[310, 80, 434.18496918678284]
[311, 80, 435.91945004463196]
[312, 80, 437.5205662250519]
[313, 80, 439.1818051338196]
[314, 80, 440.54916405677795]
[315, 80, 442.48937606811523]
[316, 80, 444.0319240093231]
[317, 80, 445.7859990596771]
[318, 80, 447.2297840118408]
[319, 80, 449.01787400245667]
[320, 80, 450.43334007263184]
[32

In [2]:
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike series
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)

501


In [27]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no4.txt','w') 
data1=[]
for k in [0..100]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"
    

done


In [29]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no5.txt','w') 
data1=[]
for k in [101..150]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [30]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no6.txt','w') 
data1=[]
for k in [151..200]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [33]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no7.txt','w') 
data1=[]
for k in [201..250]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [34]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no8.txt','w') 
data1=[]
for k in [251..300]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [35]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no9.txt','w') 
data1=[]
for k in [301..350]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [36]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no10.txt','w') 
data1=[]
for k in [351..400]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [37]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no11.txt','w') 
data1=[]
for k in [401..450]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [38]:
# break up '31aug21no1.txt' into several files
# in order to fit them onto github.
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
wfile1=open('/Users/barrybrent/31aug21no12.txt','w') 
data1=[]
for k in [451..500]:
    data1=data1+[s[k]]
t = pickle.dumps(str(data1))
wfile1.write(t)
wfile1.close()
print "done"

done


In [5]:
import pickle
rfile = open('/Users/barrybrent/31aug21no1.txt','r') # Hlambda4strike serie
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
polydata=[]
import pickle
import time
rfile = open('/Users/barrybrent/31aug21no2.txt','w') #Hlambda4strike polynomials
start=time.time()
for n in [0..80]:
    data=[]
    for k in [0..len(s)-1]:
        m = s[k][0]
        poly=s[k][1]
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,m^(3*n)*cf]]
        rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n,rl]]
    finish=time.time()
    print([n,rl.degree(),finish-start])
t = pickle.dumps(str(polydata))
rfile.write(t) 
rfile.close()

[0, 0, 60.48898386955261]
[1, 6, 122.54075002670288]
[2, 12, 189.54458808898926]
[3, 18, 256.0236020088196]
[4, 24, 323.17727398872375]
[5, 30, 393.60103702545166]
[6, 36, 456.0836338996887]
[7, 42, 518.2399079799652]
[8, 48, 581.1468250751495]
[9, 54, 644.9233939647675]
[10, 60, 709.492604970932]
[11, 66, 777.1252000331879]
[12, 72, 844.4778969287872]
[13, 78, 912.3603510856628]
[14, 84, 981.1546099185944]
[15, 90, 1051.7972249984741]
[16, 96, 1124.6104290485382]
[17, 102, 1199.105374097824]
[18, 108, 1273.744194984436]
[19, 114, 1352.0754790306091]
[20, 120, 1431.2368309497833]
[21, 126, 1511.3362889289856]
[22, 132, 1595.3229830265045]
[23, 138, 1684.1963839530945]
[24, 144, 1773.83447599411]
[25, 150, 1864.2861058712006]
[26, 156, 1956.776004076004]
[27, 162, 2051.185649871826]
[28, 168, 2147.739187002182]
[29, 174, 2248.6739540100098]
[30, 180, 2349.5960319042206]
[31, 186, 2456.360498905182]
[32, 192, 2566.197741985321]
[33, 198, 2675.6278851032257]
[34, 204, 2788.344805955887]
[

In [25]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    print("=====================================================")
    n=s[k][0]
    print "n: ",n
    f = D(s[k][1])
    nfc=numericalfactor(f)
    print(factor(f))
    pel=polynomialExponentList(f)
    nf=numberOfFactors(f)
    print([nf,nfc,pel])

n:  1
(16) * (x - 2) * (x + 2) * x^4
[3, 16, [1, 1, 4]]
n:  2
(112) * (x - 2) * (x + 2) * x^8 * (x^2 - 60/7)
[4, 112, [1, 1, 8, 1]]
n:  3
(448) * (x - 2) * (x + 2) * x^12 * (x^4 - 1432/63*x^2 + 7792/63)
[4, 448, [1, 1, 12, 1]]
n:  4
(1136) * (x - 2) * (x + 2) * x^16 * (x^6 - 82844/1917*x^4 + 1175920/1917*x^2 - 5270336/1917)
[4, 1136, [1, 1, 16, 1]]
n:  5
(2016) * (x - 2) * (x + 2) * x^20 * (x^8 - 2850704/42525*x^6 + 76286944/42525*x^4 - 892702976/42525*x^2 + 1218080512/14175)
[4, 2016, [1, 1, 20, 1]]


In [68]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    print("=====================================================")
    n=s[k][0]
    f = D(s[k][1])
    pel=polynomialExponentList(f)
    nfc=numericalfactor(f)
    dividend=f;divisor=D(nfc*(x-2)*(x+2)*x^(4*n))
    dv=division(dividend,divisor)
    dv0=D(dv[0])
    tf=dv0.is_irreducible()
    print(k,n,nfc,pel,tf)
    print "dv0: ",dv0
    print(factor(f))
    print dv

(1, 1, 16, [1, 1, 4], False)
dv0:  1
(16) * (x - 2) * (x + 2) * x^4
(1, 0)
(2, 2, 112, [1, 1, 8, 1], True)
dv0:  x^2 - 60/7
(112) * (x - 2) * (x + 2) * x^8 * (x^2 - 60/7)
(x^2 - 60/7, 0)
(3, 3, 448, [1, 1, 12, 1], True)
dv0:  x^4 - 1432/63*x^2 + 7792/63
(448) * (x - 2) * (x + 2) * x^12 * (x^4 - 1432/63*x^2 + 7792/63)
(x^4 - 1432/63*x^2 + 7792/63, 0)
(4, 4, 1136, [1, 1, 16, 1], True)
dv0:  x^6 - 82844/1917*x^4 + 1175920/1917*x^2 - 5270336/1917
(1136) * (x - 2) * (x + 2) * x^16 * (x^6 - 82844/1917*x^4 + 1175920/1917*x^2 - 5270336/1917)
(x^6 - 82844/1917*x^4 + 1175920/1917*x^2 - 5270336/1917, 0)
(5, 5, 2016, [1, 1, 20, 1], True)
dv0:  x^8 - 2850704/42525*x^6 + 76286944/42525*x^4 - 892702976/42525*x^2 + 1218080512/14175
(2016) * (x - 2) * (x + 2) * x^20 * (x^8 - 2850704/42525*x^6 + 76286944/42525*x^4 - 892702976/42525*x^2 + 1218080512/14175)
(x^8 - 2850704/42525*x^6 + 76286944/42525*x^4 - 892702976/42525*x^2 + 1218080512/14175, 0)


In [69]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..5]:
    #print("=====================================================")
    n=s[k][0]
    f = D(s[k][1])
    pel=polynomialExponentList(f)
    nfc=numericalfactor(f)
    dividend=f;divisor=D(nfc*(x-2)*(x+2)*x^(4*n))
    dv=division(dividend,divisor)
    dv0=D(dv[0])
    tf=dv0.is_irreducible()
    data=data+[tf]
    #print(k,n,nfc,pel,tf)
    #print "dv0: ",dv0
    #print(factor(f))
    #print dv
print data

[False, True, True, True, True]


In [70]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    #print("=====================================================")
    n=s[k][0]
    f = D(s[k][1])
    pel=polynomialExponentList(f)
    nfc=numericalfactor(f)
    dividend=f;divisor=D(nfc*(x-2)*(x+2)*x^(4*n))
    dv=division(dividend,divisor)
    dv0=D(dv[0])
    tf=dv0.is_irreducible()
    data=data+[tf]
    #print(k,n,nfc,pel,tf)
    #print "dv0: ",dv0
    #print(factor(f))
    #print dv
print data

[False, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True, True]


In [4]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
len(s)

81

In [6]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..10]:
    n=s[k][0]
    f = D(s[k][1])
    dg=f.degree()
    print (n,dg-6*n)
    
    

(1, 0)
(2, 0)
(3, 0)
(4, 0)
(5, 0)
(6, 0)
(7, 0)
(8, 0)
(9, 0)
(10, 0)


In [7]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    dg=f.degree()
    diff=dg-(6*n)
    data=data+[diff]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [17]:
def divisorsum(n):
    dvs=divisors(n)
    sm=0
    for k in [0..len(dvs)-1]:
        d=dvs[k]
        sm=sm+d^3*(-1)^(n-d)
    return 16*sm
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
data=[]
for k in [1..len(s)-1]:
    n=s[k][0]
    f = D(s[k][1])
    nfc=numericalfactor(f)
    diff=nfc-divisorsum(n)
    data=data+[diff]
print data

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [18]:
import pickle
rfile = open('/Users/barrybrent/31aug21no2.txt','r') #Hlambda4strike polynomials
cn4rfile = open('/Users/barrybrent/10aug21no2.txt','r') #H4 polynomials
rs=pickle.load(rfile)
cn4rs=pickle.load(cn4rfile)
rfile.close()
cn4rfile.close()
s=stripQuotationMarks(rs)
cn4s=stripQuotationMarks(cn4rs)
print (len(s),len(cn4s))

(81, 101)
